# Landsat-8


<div class="alert-info">

### Overview
    
* **teaching:** 30 minutes
* **exercises:** 0
* **questions:**
    * How can I find, anaylize, and visualize Landsat8 satellite imagery for an area of interest using Python?
    
</div>


This notebook will focus on accessing public datasets on AWS for a target area affected by Cyclone Kenneth (2019-04-25). Read more about this event and its impact at the [Humanitarian Open Street Map website](https://tasks.hotosm.org/project/5977). We will use a bounding box we will work with covers the island of Nagazidja, including the captial [city of Moroni](https://en.wikipedia.org/wiki/Moroni,_Comoros) - Union of the Comoros, a sovereign archipelago nation in the Indian Ocean. 

We will examine raster images from the [Landsat-8 instrument](https://www.usgs.gov/land-resources/nli/landsat). The Landsat program is the longest-running civilian satellite imagery program, with the first satellite launched in 1972 by the US Geological Survey. Landsat 8 is the latest satellite in this program, and was launched in 2013. Landsat observations are processed into “scenes”, each of which is approximately 183 km x 170 km, with a spatial resolution of 30 meters and a temporal resolution of 16 days. The duration of the landsat program makes it an attractive source of medium-scale imagery for land surface change analyses.

Additional code examples for Landsat-8 can be found in Geohackweek 2018 content: https://geohackweek.github.io/raster/04-workingwithrasters/

## Table of contents

1. [**Sat-search**](#Sat-search)
1. [**Holoviz visualization**](#Holoviz)
1. [**Rasterio and xarray**](#Rasterio-and-xarray)

In [ ]:
# Import libraries
import geopandas as gpd
import pandas as pd
import satsearch
from satstac import Items

import holoviews as hv
import hvplot.xarray
import hvplot.pandas
import geoviews as gv

import ipywidgets
import datetime

from ipywidgets import interact
from IPython.display import display, Image

import json
from cartopy import crs as ccrs

import rasterio
import rasterio.mask
from rasterio.session import AWSSession
import xarray as xr

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline

In [ ]:
# Set up our bounding box
bbox = [43.16, -11.32, 43.54, -11.96]
west, north, east, south = bbox
bbox_ctr = [0.5*(north+south), 0.5*(west+east)]

## Sat-search 

[Sat-search](https://github.com/sat-utils/sat-search) is open-source software designed to easily discover public imagery on AWS. It depends upon metadata called Spatio-Temporal Asset Catalogs [STAC catalogs](https://stacspec.org/) to filter scenes. We will use it to search for Landsat-8 data covering our area of interest

In [ ]:
# bbox as a python list is great for use in python, but we can instead save to a more interoperable format (GeoJSON)
# Here is a great website for creating and visualizing geojson on a map: http://geojson.io

aoi = { "type": "Polygon", 
    "coordinates": [[[west, south], [west, north], [east, north], [east, south], [west, south]]]
}
# pretty print formatting
print(json.dumps(aoi, sort_keys=False, indent=2))

# save to file for future use
with open('aoi-5977.geojson', 'w') as f:
    json.dump(aoi, f)

In [ ]:
# Load results to pandas geodataframe
# now other packages such as geojson can read this file
gfa = gpd.read_file('aoi-5977.geojson')
gfa

In [ ]:
# Get results for bbox and time range
results = satsearch.Search(bbox=bbox, datetime='2019-02-01/2019-06-01')
print('%s items' % results.found())
items = results.items()
print('%s collections:' % len(items._collections))
print(items._collections)


In [ ]:
# If you are unfamiliar with one of these satellites, we can look at stored metadata
col = items._collections[1]

print('Title:', col.title)
print('Collection Version:', col.version)
print('Keywords: ', col.keywords)
print('License:', col.license)
print('Providers:', col.providers)
print('Extent', col.extent)

In [ ]:
# We can delve deeper to see what kind of metadata is available at the scene level
for key in col.properties:
    if key == 'eo:bands':
        [print(band) for band in col[key]]
    else:
        print('%s: %s' % (key, col[key]))

In [ ]:
# Search for just tier1 Landsat8 scenes, all dates
properties =  ["landsat:tier=T1"] 

bbox = (west, south, east, north) #(min lon, min lat, max lon, max lat)

results = satsearch.Search.search(collection='landsat-8-l1', 
                        bbox=bbox, 
                        sort=['<datetime'], #earliest scene first
                        property=properties)
print('%s items' % results.found())

In [ ]:
# Save search results for later or to share with others
items = results.items()
items.save('items-landsat8.json')
items = Items.load('items-landsat8.json')

In [ ]:
# # Assets correspond to actual images related to a STAC metadata item
# Use pandas to better display python dictionaries!
pd.DataFrame(items[0].assets).T.reset_index()

In [ ]:
# Read results into a geopandas GeoDataFrame
gfl = gpd.read_file('items-landsat8.json')
gfl = gfl.sort_values('datetime').reset_index(drop=True)
print('records:', len(gfl))
gfl.head()

In [ ]:
# Hack for neat display of band information
import ast
band_info = pd.DataFrame(ast.literal_eval(gfl.iloc[0]['eo:bands']))
band_info

In [ ]:
# Note the cloud_cover column, we can narrow our search by any of these properties
properties.extend(["eo:cloud_cover<10"])

test = satsearch.Search.search(collection='landsat-8-l1', 
                        bbox=bbox, 
                        sort=['<datetime'], #earliest scene first
                        property=properties)
print('%s items' % test.found())

In [ ]:
# Or since we can just use geopandas to filter results
subset = gfl[gfl['eo:cloud_cover'] < 10]
print('%s items' % len(subset))

## Holoviz

[Holoviz](https://holoviz.org/) is a set of Python visualization libraries that simplify interactive visualizations of data in a web-browser. We'll use several of these libraries including hvplot and geoviews to visualize both vector data (such as image footprints) and raster data (actual raster values). 

<div class="alert-warning">

#### Note 
    
the toolbars on the right and side of these plots. We are using a library called Bokeh that gives interactive widgets to zoom in and pan around on maps.
</div>

In [ ]:
# Plot search AOI and frames on a map using Holoviz Libraries
cols = gfl.loc[:,('id','geometry')]

footprints = cols.hvplot(geo=True, line_color='k', alpha=0.1, title='Landsat 8 T1')
aoi = gfa.hvplot(geo=True, line_color='b', fill_color=None)
tiles = gv.tile_sources.CartoEco.options(width=700, height=500) 
labels = gv.tile_sources.StamenLabels.options(level='annotation')
tiles * footprints * aoi * labels

## ipywidgets

[ipywidgets](https://ipywidgets.readthedocs.io/en/latest/) provide another convenient approach to custom visualizations. The function below allows us to browse through all the image thumbnails for a group of images (more specifically a specific Landsat8 path and row). 

In [ ]:
def browse_images(items):
    n = len(items)

    def view_image(i=0):
        item = items[i]
        print(f"id={item.id}\tdate={item.datetime}\tcloud%={item['eo:cloud_cover']}")
        display(Image(item.asset('thumbnail')['href']))
    
    interact(view_image, i=(0,n-1))

In [ ]:
# Custom syntax (additional fields, query strings instead of query dict)
properties =  ["eo:row=068",
               "eo:column=162",
               "landsat:tier=T1"] 
results = satsearch.Search.search(collection='landsat-8-l1', 
                        bbox=bbox, 
                        sort=['<datetime'], #earliest scene first
                        property=properties)
print('%s items' % results.found())
items = results.items()

In [ ]:
# May not work on Chrome currently, does work on Safari
browse_images(items) 

## Rasterio and xarray

To actually load full resolution data from a particular Landsat-8 band we'll use rasterio and xarray libraries.

In [ ]:
# These are environmnent variable settings for efficiently reading data on AWS S3
env = rasterio.Env(GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR',
                  CPL_VSIL_CURL_ALLOWED_EXTENSIONS='TIF',
                 )

In [ ]:
item = items[0]
band = 'red'
url = item.asset(band)['href']
print(url)
with env:
    with rasterio.open(url) as src:
        print(src.profile) # image metadata
        width = src.width
        blockx = src.profile['blockxsize']
        blocky = src.profile['blockysize']
        xchunk = int(width/blockx)*blockx
        ychunk = blocky
        da = xr.open_rasterio(src, chunks={'band': 1, 'x': xchunk, 'y': ychunk})
da

In [ ]:
# This will pull raster data over network. if operating in the same AWS region, should be very fast!
# NOTE: seems there is a bug currently with 'logz' for a log-scale colorbar
img = da.hvplot.image(rasterize=True, logz=True, width=700, height=500, cmap='reds', title=f'{item.id} ({band})')

img 

### Visualize with on-the-fly reprojection

In [ ]:
# Display image in latitute, longitude coordinates instead of EPSG:32638 (UTM 38N)
crs = ccrs.UTM(zone='38N') 
img = da.hvplot.image(crs=crs, rasterize=True, width=700, height=500, cmap='reds', alpha=0.8, title=f'{item.id} ({band})') # , logz=True not working 
aoi = gfa.hvplot(geo=True, line_color='b', fill_color=None)
img * aoi

### Image subsets and crop by shapefile

Often we are only interested in small regions of full images. One of the killer features of cloud-optimized data formats stored on the cloud is that we can efficiently pull subsets of an image rather than the whole thing. Here we'll pull only the pixels within a vector polygon in our area of interest.

<div class="alert-warning">

#### Note 
    
It's up to you to make sure the vector and raster CRS's match!
</div>

In [ ]:
with rasterio.open(url) as src:
    # re-project vector to match raster CRS
    print(src.meta)
    shape = gfa.to_crs(epsg=src.crs.to_epsg())
    out_image, out_transform = rasterio.mask.mask(src, shape.geometry.values, crop=True)
    out_meta = src.meta
    out_meta.update({
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
    print(out_meta)


In [ ]:
# Excercise 1) Load and visualize the highest-resolution 15m pancromatic band instead of the red band
# Excercise 2) Calculate a band ratio between any two bands